In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize

from statsmodels.sandbox.regression.gmm import IV2SLS 
# There is a package named IV2SLS in Python. Do not use this package! The exogenous explanatory variables must
# be entered as instruments. So it gives wrong answers
from statsmodels.sandbox.regression.gmm import GMM

# PART 1

1. Update the GMM model that we discussed in class by incorporating the δ term to the instrumental-variable moment expressions.

In [2]:
# Load the data
input_table = pd.read_csv('https://raw.githubusercontent.com/mn42899/predictive_modelling/refs/heads/main/midterm_partone.csv')
input_table

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446
...,...,...,...,...,...,...,...,...
1691,1,-0.015543,5.225766,0.309119,1.615384,3.554503,2.197871,0.005549
1692,1,0.399089,5.324390,0.274782,1.463044,3.745006,2.324502,0.004359
1693,1,-0.702200,5.575258,0.287503,1.602905,3.434909,2.282626,0.000000
1694,1,0.283926,5.423463,0.256657,1.391968,2.876645,1.454948,0.000000


In [3]:
# original model
y_vals = np.array(input_table["Stock Change"])
x_vals = np.array(input_table[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])
n_obs = len(y_vals)

class og_gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = og_gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                og_gmm Results                                
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                         og_gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Thu, 07 Nov 2024                                         
Time:                        19:53:41                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

In [ ]:
# Updated model to see if there is bias
class biased_gmm(GMM):
    def _init_(self, endog, exog, instrument):
        super()._init_(endog, exog, instrument)

    def momcond(self, params):
        # Separate out delta from the other parameters
        beta = params[:-1]  # First four parameters
        delta = params[-1]  # Last parameter = delta
        
        endog = self.endog
        exog = self.exog
        inst = self.instrument
        
        # Basic error term
        error = endog - beta[0] - beta[1] * exog[:,0] - beta[2] * exog[:,1] - beta[3] * exog[:,2]
        
        # Complete set of moment conditions
        error0 = error  # Basic moment
        error1 = error * exog[:,1]  # Exogenous variable moments
        error2 = error * exog[:,2]
        error3 = error * inst[:,0] - delta  
        error4 = error * inst[:,1] - delta
        error5 = error * inst[:,2] - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

# Initialize and fit model
initial_params = np.array([0.1, 0.1, 0.1, 0.1, 0.0])  # Initial guess including delta
model_bias = biased_gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5)
res_bias = model_bias.fit(initial_params)

# Print results
print(res_bias.summary())

# Calculate test statistics for delta
std_errors = np.sqrt(np.diag(res_bias.cov_params()))
delta_t_stat = res_bias.params[-1] / std_errors[-1]
delta_p_value = 2 * (1 - stats.norm.cdf(abs(delta_t_stat)))

print("Delta Outputs:")
print("=" * 50)
print(f"Estimated δ: {res_bias.params[-1]:.6f}")
print(f"Standard Error: {std_errors[-1]:.6f}")
print(f"Z-statistic: {delta_t_stat:.4f}")
print(f"P-value: {delta_p_value:.4f}")

Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 12
         Function evaluations: 16
         Gradient evaluations: 16
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5
                              biased_gmm Results                              
Dep. Variable:                      y   Hansen J:                       0.5862
Model:                     biased_gmm   Prob (Hansen J):                 0.444
Method:                           GMM                                         
D

2.	By analyzing the GMM summary table and test statistics of coefficients, determine if the industry expert’s claim is statistically justified.

The industry expert's claim is somewhat statistically justified. Since the expert's claim is that bias is present if delta is a non-zero value, there is some credence to what was said with a value seen above at -0.0006. However, because this is incrementally below 0, the bias present is relatively small so the expert's claim is only minimally justified. This is further explained in the report

# PART 2

1.	Divide the dataset equally into two as training (50%) and test (50%) sets. Use the training set to fit a logistic regression model, where the credit rating is the dependent variable. Apply the model to the test set, and report the confusion matrix, recall, precision, and F1 score values.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load the dataset
url = 'https://raw.githubusercontent.com/mn42899/predictive_modelling/refs/heads/main/midterm_parttwo.csv'
data = pd.read_csv(url)
data.head()

In [ ]:
data['Credit Rating'].value_counts()

In [ ]:
# Convert categorical variables to numerical (if needed)
data['Credit Rating'] = data['Credit Rating'].apply(lambda x: 1 if x == 'Positive' else 0)

# Define features and target variable
X = pd.get_dummies(data.drop('Credit Rating', axis=1), drop_first=True)
y = data['Credit Rating']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate and display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Print the classification report
print("Classification Report:\n", classification_report(y_test, y_pred))

2. Suppose that the bank decided to make the credit approval process more challenging such that only 15% of the applications would be granted. Calculate the threshold value for the prediction probability, so only 15% of the test set would get their applications approved. Then, update your confusion matrix, recall, precision, and F1 scores. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

# Predict probabilities instead of labels
y_proba = model.predict_proba(X_test)[:, 1]

# Calculate threshold to classify only 15% as approved
threshold = np.percentile(y_proba, 85)

# Predict with the new threshold
y_pred_new = (y_proba >= threshold).astype(int)

# Calculate metrics with the new threshold
conf_matrix_new = confusion_matrix(y_test, y_pred_new)
precision_new = precision_score(y_test, y_pred_new)
recall_new = recall_score(y_test, y_pred_new)
f1_new = f1_score(y_test, y_pred_new)

# Display Confusion Matrix using Seaborn
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_new, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix (15% Threshold)")
plt.show()

# Display Classification Report
print("Classification Report with 15% Threshold:\n")
print(classification_report(y_test, y_pred_new))

# Also store the results in variables if needed
print("\nMetrics with 15% Threshold:")
print("Precision:", precision_new)
print("Recall:", recall_new)
print("F1 Score:", f1_new)